In [1]:
import torch
import torch.nn as nn


In [ ]:
input = torch.randn(2,3,4, requires_grad=True)

print(input)
print(input.mean(-1, keepdim=True))
print(input.mean(1))

variance_epsilon = 1e-6


In [ ]:
input = input.to(torch.float32)
variance = input.pow(2).mean(-1, keepdim=True)
hidden_states = input * torch.rsqrt(variance + variance_epsilon)
print(0.1089**2, input.pow(2))

layerNorm = nn.RMSNorm([4])
hidden_states1 = layerNorm(input)
print(hidden_states)
print(hidden_states1)

In [ ]:
class SinusodialPE(nn.Module):
    def __init__(self, base=10000) -> None:
        super().__init__()
        
        self.dim = 128
        self.base = base        
        self.max_seq_len = 5
        
        self.pe = torch.zeros(self.max_seq_len, self.dim) # shape = [max_seq_len, dim]
        
        pos = torch.arange(0, self.max_seq_len, dtype=torch.float) # shape = [max_seq_len,]
        print(pos.shape)
        theta = 1 / self.base ** (torch.arange(0, self.dim, 2).float()  / self.dim)#.to(config.device) / self.dim) # shape = [dim//2,]
        print(torch.arange(0, self.dim, 2))
        
        half_pe = torch.outer(pos, theta) # shape = [max_seq_len, dim//2]
        print(half_pe)
        self.pe[:, 0::2] = torch.sin(half_pe)
        self.pe[:, 1::2] = torch.cos(half_pe)
        
    def forward(self, position_ids):
        return self.pe[position_ids]


sinpe = SinusodialPE()
sinpe.pe.shape

In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F


range: tuple = (-5,5)
x = torch.arange(range[0], range[1], 0.1)
plt.plot(x, F.silu(x), label='silu')

In [ ]:
class FFN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        
        self.hidden_size = 2048
        self.ffn_size = 8192
        self.dropout_p = 0.05
        
        self.up_proj = nn.Linear(self.hidden_size, self.ffn_size)
        self.down_proj = nn.Linear(self.ffn_size, self.hidden_size)#.to(config.device)
        
        # self.activation_func = F.relu
        # self.activation_func = F.elu
        self.activation_func = F.gelu
        self.dropout = nn.Dropout(p=self.dropout_p)
        
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        """

        Args:
            hidden_states (torch.Tensor): shape: [batch_size, seq_len, hidden_size]

        Returns:
            torch.Tensor: shape: [batch_size, seq_len, hidden_size]
        """
        hidden_states = self.up_proj(hidden_states) # [batch_size, seq_len, ffn_size]
        hidden_states = self.activation_func(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.down_proj(hidden_states) # [batch_size, seq_len, hidden_size]
        
        return hidden_states
        
ffn = FFN()
ffn